<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JOIN-tables" data-toc-modified-id="JOIN-tables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JOIN tables</a></span></li><li><span><a href="#Считаем-таргет-как-pr_sales_in_units-+-pr_promo_sales_in_units" data-toc-modified-id="Считаем-таргет-как-pr_sales_in_units-+-pr_promo_sales_in_units-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Считаем таргет как pr_sales_in_units + pr_promo_sales_in_units</a></span></li><li><span><a href="#Соответственно,-их-удаляем,-оставляем-только-сумму-как-таргет" data-toc-modified-id="Соответственно,-их-удаляем,-оставляем-только-сумму-как-таргет-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Соответственно, их удаляем, оставляем только сумму как таргет</a></span></li><li><span><a href="#Также-удаляем-суммы-в-руб" data-toc-modified-id="Также-удаляем-суммы-в-руб-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Также удаляем суммы в руб</a></span></li><li><span><a href="#добавляем-в-тест-признак-промо-(=0)" data-toc-modified-id="добавляем-в-тест-признак-промо-(=0)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>добавляем в тест признак промо (=0)</a></span></li><li><span><a href="#делим-на-признаки-и-таргет" data-toc-modified-id="делим-на-признаки-и-таргет-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>делим на признаки и таргет</a></span></li><li><span><a href="#Кодируем-признаки-и-обучаем" data-toc-modified-id="Кодируем-признаки-и-обучаем-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Кодируем признаки и обучаем</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

In [2]:
#загрузка данных
try:
    sales_df_train = pd.read_csv("sales_df_train.csv")
    sales_submission = pd.read_csv("sales_submission.csv")
    pr_df = pd.read_csv("pr_df.csv")
    st_df = pd.read_csv("st_df.csv")
except:
    print('Проверить путь к файлам')

## JOIN tables

In [3]:
df_train = sales_df_train.merge(pr_df, how='left', on='pr_sku_id')
df_train = df_train.merge(st_df, how='left', on='st_id')

df_test = sales_submission.merge(pr_df, how='left', on='pr_sku_id')
df_test = df_test.merge(st_df, how='left', on='st_id')

## Считаем таргет как pr_sales_in_units + pr_promo_sales_in_units

In [4]:
df_train['target'] = df_train['pr_sales_in_units'] + df_train['pr_promo_sales_in_units']

## Соответственно, их удаляем, оставляем только сумму как таргет
## Также удаляем суммы в руб

In [5]:
df_train = df_train.drop(columns=['pr_sales_in_units', 
                                  'pr_promo_sales_in_units','pr_sales_in_rub', 'pr_promo_sales_in_rub'])

## добавляем в тест признак промо (=0)

In [6]:
df_test['pr_sales_type_id'] = 0

## делим на признаки и таргет

In [7]:
X_train = df_train.drop(columns=['target'])
y_train = df_train['target']

X_test = df_test.drop(columns=['target'])
y_test = df_test['target']

In [8]:
cc = df_train.columns.to_list()
cc.sort()
cc.pop()

#сортируем колонки, чтобы сходились
X_train = X_train.reindex(columns=cc)
X_test = X_test.reindex(columns=cc)

X_train['date'] = pd.to_datetime(X_train['date']).values.astype("float64")
X_test['date'] = pd.to_datetime(X_test['date']).values.astype("float64")

## Кодируем признаки и обучаем

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=9999)

#code features
col = ['pr_cat_id', 'pr_group_id', 'pr_sku_id', 'pr_subcat_id', 
       'st_city_id', 'st_division_code', 'st_id']
X_train[col] = oe.fit_transform(X_train[col])
X_test[col] = oe.transform(X_test[col])

#initialise the model
model = LinearRegression()

#fit the model on train data
model.fit(X_train, y_train)

#make predictions
y_pred = model.predict(X_test)